# Assignment 4

## Main tasks

In [8]:
from utils import fk_symb
import sympy as sp

# (4,4)
Tbase = sp.eye(4)
# (7,4,4)
Ts_q = [Tbase] + fk_symb

In [9]:
from utils import qs_symb, n_link
t = sp.symbols('t')

qs = [sp.Function(f'q{i}')(t) for i in range(1,n_link+1)]
Ts = [t.subs(zip(qs_symb,qs)) for t in Ts_q]

In [10]:
import numpy as np
# (7,3)
ps = [np.array(t[:3,3].T.tolist())[0] for t in Ts]

In [11]:
from utils import link_lengths
# (6,1)
cs = link_lengths / 2

In [12]:
import numpy as np
# (6,1)
ks = np.array([2,0,0,2,0,2])

In [13]:
# (6,3,3)
Rs = [t[:3,:3] for t in Ts[1:]]
# (6,3)
pcs = [np.array((t[:3,3] + c * t[:3,k]).tolist()) for t,c,k in zip(Ts[1:], cs, ks)]
# (6,4,4)
Tcs = [sp.eye(4) for i in pcs]
for i,r,p in zip(Tcs, Rs, pcs):
    i[:3,:3] = r
    i[:3,3] = p

In [14]:
# (7,1,3)
us = [np.array(r[:3,2].T.tolist())[0] for r in Ts]

In [55]:
r = 1
# (6,)
ms = [1,3,5,2,4,6]
# (6,)
axs = ['x','z','z','x','z','x']

def get_inertia_matrix(idx, axis):
    i1 = 1/12 * ms[idx] * (3 * r**2 + link_lengths[idx]**2)
    i2 = 1/2 * ms[idx] * r**2

    if axis == 'z':
        return np.diag([i1,i1,i2])
    elif axis == 'x':
        return np.diag([i2,i1,i1])
    else:
        return None

# (6,3,3)
Is = [get_inertia_matrix(i,ax) for i,ax in zip(range(6),axs)]

In [56]:
from utils import n_link

# (6,3,6)
Jvs = [None] * n_link
# (6,3,6)
Jws = [None] * n_link
# (6,3)
pcs1 = [pc[0] for pc in pcs]

for i in range(n_link):
    Jv = sp.zeros(3,6)
    Jw = sp.zeros(3,6)
    for k in range(i+1):
        Jv[:3,k] = np.cross(us[k], pcs1[i] - ps[k])
        Jw[:3,k] = us[k]
    
    Jvs[i] = Jv
    Jws[i] = Jw

In [17]:
M = sp.zeros(6,6)
for m,jv,jw,r,i in zip(ms,Jvs,Jws,Rs,Is):
    M += m * jv.T * jv + jw.T * r * i * r.T * jw

In [73]:
from sympy.utilities import lambdify
M_num = lambdify(tuple(qs),M,'numpy')

In [74]:
np.set_printoptions(precision=4)
print(M_num(*[0,1,0,1,0,1]))

[[ 41.9553   2.5836  -6.9394  -0.8001  -9.4994  20.2535]
 [  2.5836  33.2065  27.5389  -4.2521 -16.664   -4.0442]
 [ -6.9394  27.5389  42.4496  -4.2521 -23.9377 -14.7718]
 [ -0.8001  -4.2521  -4.2521  77.819   34.0796   0.0383]
 [ -9.4994 -16.664  -23.9377  34.0796  46.3123  -0.0374]
 [ 20.2535  -4.0442 -14.7718   0.0383  -0.0374  41.32  ]]


In [28]:
Mdot = sp.diff(M,t)

In [37]:
C = sp.zeros(n_link, n_link)
q_dots = [sp.Symbol(f'qd{i}') for i in range(1,n_link+1)]

for i in range(n_link):
    for j in range(n_link):
        for k in range(n_link):
            C[i, j] += (
                1
                / 2
                * (
                    M[i, j].diff(qs[k])
                    + M[i, k].diff(qs[j])
                    - M[j, k].diff(qs[i])
                )
                * q_dots[k]
            )

In [39]:
C_num = lambdify(tuple([tuple(qs),tuple(q_dots)]),C,'numpy')

In [41]:
q1 = [0,1,0,1,0,1]
q2 = [1,0,1,0,1,0]
C_num(q1,q2)

array([[-44.461 , -64.43  , -28.8308, -17.9399,  12.9534,  -8.3808],
       [ 47.4361,   2.4878, -11.0268, -11.213 ,   0.1017,  10.3985],
       [ 46.6385,  47.1607,  35.6166,  23.9306, -10.6997,   1.2679],
       [ 10.577 ,  28.5088,  -1.5239,  60.9424,  -6.4566, -13.9125],
       [  4.6757,  -9.1744, -29.3278,  -1.9536,  -2.4099,   6.9524],
       [-42.6622, -45.5654, -35.0067, -62.0361,  26.8044,   9.0469]])

In [57]:
Jvs_num = [lambdify(tuple(qs), j) for j in Jvs]

In [80]:
from sympy.utilities.lambdify import implemented_function
g = 9.81
g0 = np.array([0,0,g])


def G_num(qs):
    g = np.zeros(n_link)
    for j,m in zip(Jvs_num,ms):
        g += (j(*qs).T.dot(g0)) * m
    return -g

G_num(q1)

array([  -0.    , -140.2443,  -26.2864,   42.1897,   36.9794,   19.4978])

In [79]:
def get_torques(qs,q_dots,q_ddots):
    return M_num(*qs).dot(q_ddots) + C_num(qs,q_dots) + G_num(qs)